In [1]:
import numpy as np
import numpy.linalg as LA
import meshplot as mp
from IPython.display import JSON as DJSON
from IPython.display import clear_output
from pspart import Part
from pspart import NNHash
import os
import pandas as ps
from mate_proposals import mate_proposals, homogenize_frame
from scipy.spatial.transform import Rotation as R
import meshplot as mp
import onshape.brepio as brepio

In [2]:
datapath = '/projects/grail/benjones/cadlab'

In [3]:
name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
assembly_df = ps.read_hdf(name,'assembly')
mate_df = ps.read_hdf(name,'mate')
part_df = ps.read_hdf(name,'part')
mate_df['MateIndex'] = mate_df.index

In [4]:
mate_df.set_index('Assembly', inplace=True)
part_df.set_index('Assembly', inplace=True)    

In [5]:
with open('fully_connected_moving_no_multimates.txt','r') as f:
    set_E_indices = [int(l.rstrip()) for l in f.readlines()]

In [31]:
epsilon_rel = 0.001

invalids = []
valids = []
invalid_counts = []
for ind in set_E_indices[:10]:
    #clear_output(wait=True)
    #display(f'num_processed: {ind}/{assembly_df.shape[0]}')

    #1. spatially hash all MCFs (cache hash maps for each part for re-use with individual mates)
    #2. for all mates, ensure that each MCF is represented (keep track of closest/equivalent MCFs, log percentage of assemblies for which this holds)
    #3. get proposals, edit appropriate ones to true based on equivalence class computed per mated pair of parts (taking outer product of equivalent MCs on left and right)

    print(f'assembly {assembly_df.loc[ind,"AssemblyPath"]}')
    
    part_subset = part_df.loc[ind]
    mate_subset = mate_df.loc[ind]
    if mate_subset.ndim == 1:
        mate_subset = ps.DataFrame([mate_subset], columns=mate_subset.keys())
    
    parts = []
    transforms = []
    mcf_hashes = []
    mc_frames_all = []
    occ_to_index = dict()
    
    #debug
    #all_points = []
    
    for j in range(part_subset.shape[0]):
        path = os.path.join(datapath, 'data/models', *[part_subset.iloc[j][k] for k in ['did','mv','eid','config']], f'{part_subset.iloc[j]["PartId"]}.xt')
        assert(os.path.isfile(path))
        part = Part(path)
        parts.append(part)
        tf = part_subset.iloc[j]['Transform']
        transforms.append(tf)
        occ_to_index[part_subset.iloc[j]['PartOccurrenceID']] = j
        
    maxdim = max([(part.bounding_box()[1]-part.bounding_box()[0]).max() for part in parts])
    threshold = maxdim * epsilon_rel
    
    for j in range(len(parts)):
        part = parts[j]
        tf = transforms[j]
        mc_frames = []
        for mc in part.all_mate_connectors:
            cs = mc.get_coordinate_system()
            frame = tf[:3,:3] @ cs[:3,:3]
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            origin = tf[:3,:3] @ cs[:3,3] + tf[:3,3]
            #all_points.append(origin)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_frames.append(np.concatenate([origin, rot]))
        mc_frames_all.append(mc_frames)
        frame_hash = NNHash(mc_frames, 7, threshold)
        #frame_hash = NNHash([mc_frame[:3] for mc_frame in mc_frames], 3, threshold)
        mcf_hashes.append(frame_hash)
    
    #all_points = np.array(all_points)
    #p = mp.plot(all_points)
    print(f'threshold: {threshold}')
    num_invalid_mates = 0
    for j in range(mate_subset.shape[0]):
        for i in range(2):
            occId = mate_subset.iloc[j][f'Part{i+1}']
            partIndex = occ_to_index[occId]
            assert(part_subset.iloc[partIndex]['PartOccurrenceID'] == occId)
            origin_local = mate_subset.iloc[j][f'Origin{i+1}']
            frame_local = mate_subset.iloc[j][f'Axes{i+1}']
            tf = transforms[partIndex]
            origin = tf[:3,:3] @ origin_local + tf[:3,3]            
            frame = tf[:3,:3] @ frame_local
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_frame = np.concatenate([origin, rot])
            found=False
            minDist = np.inf
            closest = -1
            for k in range(len(mc_frames_all[partIndex])):
                dist = LA.norm(mc_frames_all[partIndex][k] - mc_frame)
                if dist < minDist:
                    minDist = dist
                    closest = k
                if dist < threshold:
                    found=True
                    break
            
            #neighbors = mcf_hashes[occ_to_index[occId]].get_nearest_points(np.concatenate([origin, rot]))
            #neighbors = mcf_hashes[occ_to_index[occId]].get_nearest_points(origin)
            print(f'{i} closest dist (fraction of maxdim): {minDist/maxdim}')
            invalid = dict()
            invalid['mc_origin'] = mc_frames_all[partIndex][k][:3]
            invalid['mc_frame'] = R.from_quat(mc_frames_all[partIndex][k][3:]).as_matrix() 
            invalid['mate_origin'] = origin
            invalid['mate_frame'] = frame_homogenized
            invalid['maxdim'] = maxdim
            if not found:
                num_invalid_mates += 1
                invalids.append(invalid)
                #p.add_points(origin[np.newaxis],shading={'point_color':'blue','point_size':0.02})
                print(f'invalid mate {mate_subset.iloc[j]["MateIndex"]}') 
                break
            else:
                valids.append(invalid)
    invalid_counts.append((num_invalid_mates, mate_subset.shape[0]))
    print(f'num invalid mates: {num_invalid_mates}/{mate_subset.shape[0]}')

assembly 0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88
threshold: 0.00027400000000000005
0 closest dist (fraction of maxdim): 9.443443894046388e-18
1 closest dist (fraction of maxdim): 9.44344390245686e-18
0 closest dist (fraction of maxdim): 4.051593103999865e-09
1 closest dist (fraction of maxdim): 4.1059994149735055e-09
0 closest dist (fraction of maxdim): 8.480413991459999e-09
1 closest dist (fraction of maxdim): 8.480414045052761e-09
0 closest dist (fraction of maxdim): 2.187310510115392e-09
1 closest dist (fraction of maxdim): 0.0007867660153516282
0 closest dist (fraction of maxdim): 6.526056003383406e-09
1 closest dist (fraction of maxdim): 2.1763090298212494e-08
0 closest dist (fraction of maxdim): 1.753374997089074e-08
1 closest dist (fraction of maxdim): 0.4425376255173872
invalid mate 381
0 closest dist (fraction of maxdim): 5.219986263889804e-09
1 closest dist (fraction of maxdim): 0.001277365055744381
invalid mate 382
0 closest dist (fraction 

In [33]:
invalid_counts_old = invalid_counts

In [13]:
from visualize import add_axis

In [26]:
invalid = invalids[6]
p = mp.plot(np.vstack([invalid['mc_origin'], invalid['mate_origin']]))
add_axis(p, invalid['mc_origin'], invalid['mc_frame'][:,0], invalid['mc_frame'][:,1], invalid['mc_frame'][:,2], scale=invalid['maxdim']/10)
add_axis(p, invalid['mate_origin'], invalid['mate_frame'][:,0], invalid['mate_frame'][:,1], invalid['mate_frame'][:,2], scale=invalid['maxdim']/10)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.057504…

In [ ]:
list(mate_subset['MateIndex'])

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392]

In [96]:
assembly_df[assembly_df['AssemblyPath'] == '0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88']

,AssemblyPath,ConnectedComponents,RigidPieces,LonePieces,NumParts,NumBinaryPartMates,TotalMates
18,0e2de88aa724d660e4093564_52ea6bfdd317472a99efc...,1,10,0,18,17,17


In [99]:
list(mate_df.loc[18,'MateIndex'])

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392]